## Forward Mode AD


# Introduction

Automatic Differentiation allows for fast and precise derivitative computation. It is not, however, the only method for computing derivatives. 

Derivatives are a fundamental tool in mathematics with applications in a range of scientific fields.  Differentiation is core to many computational problems including finding extrema and zeroes of functions. Optimization problems such as gradient descent in neural networks depend on partial derivatives to incrementally move towards optimal solutions. Directional derivatives and the Jacobian matrix are utilized in a variety of contexts including backpropogation in training neural networks. Directional derivatives compute the change in a mult-dimensional function with a respect to any linear combination of its input parameters. 

Because of the prevalence of derivatives in computational fields, automatic differentiation focuses on developing algorithmic techniques for efficiently evaluating derivatives.


### Symbolic differentiation
Symbolic differentiation applies basic derivative rules to compute derivatives at the expression level. While precise, symbolic differentiation uses excess computational power and memory to hold the swelling equation resulting from chain rule. Symbolic differentiation computes the derivative of a function analytically, which offers high precision at the cost of execution time and exponential size in relation to the original function. Since many science applications care about the derivative evaluated at a value, we can often bypass the need for an analytic form for the derivative.

### Finite difference method
The finite difference method, on the other hand, estimates the derivative by computing the slope of the secant line, using a small change in $x$. The precision of the derivative depends on the choice of $x$; too large of a step leads to precision error but too small of a step creates rounding error.  Numerical differentiation, on the other hand, offers O(n) time computation for an n-dimensional gradient (think multi-dimensional derivative). It achieves this by estimating the derivative instead of computing it precisely. The trade-off, as with all estimations, is a reduction in accuracy due to rounding and truncation.

### Why AutoDiff?
Automatic Differentiation achieves the precision of symbolic differentiation with time complexity that matches the complexity of computing the original function. It does this by leveraging a useful fact: when computing a derivative at a specific point, we don't need the symbolic expression of the derivative. This allows us to compute our derivative via a series of computations that compute each depending on the prior computation rather than deriving the entire expression and plugging in the point at the end.

In forward mode, the graph can be abstract as computations are made on-the-fly and no storage of all prior computations is needed. In reverse-mode, on the other hand, space needs will exceed those of computing the original function as the computational graphs of derivatives will need to be stored and computed in reverse.

## Background

Atomatic differentiation (AD) is based on the property of decomposition of a function into intermediate steps based on the chain rule: $$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$
Consider the function $f(x_1, x_2) = e^{-(\sin(x_1) + \cos(x_2)}$. 

![](https://drive.google.com/uc?export=view&id=1q8eWiv6gv-ty2rv-S0OuEOqwmeVhsLgq)

The derivative is derived by chaining together the product of the outer derivative with each of the inner derivatives. This creates a computational graph, where each edge represents an elementary operation applied to an intermediary node.

Forward mode AD uses forward accumulation of these intermediary steps, traversing the chain rule in a partial ordering from the most inner part (the original inputs) working outward until the entire derivative has been evaluated. In addition, at each step we compute the derivative of the intermediate variables, called the tangent trace. This allows for us to compute the directional derivative in a single pass.

Dual numbers are convenient for differentiation since can allow us to capture both the primal and tangent trace at the same time. A dual number can be expressed as $a + b\epsilon$, where $a$ and $b$ are real numbers and $\epsilon$ is small enough such that $\epsilon^2 = 0$. The real part stores the primal trace while the dual part stores the tangent trace.

Forward mode automatic differentiation takes a seed vector which determines the direction of the derivative. This can be any linear combination of derivatives. To compute the full Jacobian using Forward Mode AD, this requires performing $m$ passes where $m$ is the number of inputs to the original function. On each pass, we calculate the derivative with respect to that direction.

In contrast, Reverse mode AD enables us to compute the full gradient in two passes. This requires storing an explicit computational graph. In the forward pass, reverse mode computes the primal trace and partial derivatives with respect to the parent node. In the reverse pass, these are reconstructed via the chain rule by accumulating the values of all child nodes.

Runtime of Forward and Reverse modes -

When a function is defined as $ \mathbb{R}^n \rightarrow \mathbb{R}^m $, Forward Mode computes the gradient with respect to the number of independent variables.
Therefore, the computational cost of using Forward Mode depends on the number of independent variables n.

Reverse Mode computes the gradient with respect to the number of outputs. Therefore, the computational cost of using Forward Mode depends on the number of outputs, m.

As a result, if there are more outputs than inputs, Forward mode is more efficient.
On the other hand, if there are more inputs than outputs, Reverse mode is more efficient.

## Software Organization 

Our initial software organization plan is to store our AutoDiff classes in a source folder, our Test Suite in a test folder, an example script in an example folder, and a setup script for installing relevant packages at the root level. This provides a clear distinction between core functionality, testing, examples, and setup. 

```
team12/
├── /docs
│   └── milestone1
├── LICENSE
├── README.md
└── /src
│   └── /core
│   │   ├── base_derivatives.py (holds common basic derivatives e.g. trig)
│   │   ├── dual_number.py (holds override Dunder methods)
│   │   └── override_funcs.py (holds basic math functions for dual numbers)
│   ├── forward.py (holds forward pass class)
│   ├── reverse.py (holds reverse pass class)
│   └── /structures
│       └── graph.py (Reverse: node class)
├── /tests
│     ├── individual test files per module
│     └── test.py (runs all tests and delivers relevant output)
├── /examples
│     └── app.py
└── setup.py (runs relevant instillations)

```
At the root level of our source folder we have the base classes that form our AutoDiff implementation. Core will hold a class that includes functions that compute primitive derivatives and a dual number class with its dunder methods overriden to work as dual numbers. 

In adition to core, the source folder holds two AD method classes, forward and reverse. For reverse, we need to store the computational graph and nodes and therefore these are stored in a structures folder at the same level as reverse.

### Distribution

For the final release, we will distribute autodiffpypi via PyPI with PEP517/518. The current state uses a simple `requirements.txt` file to manage dependencies.


## Implementation Details

This section provides a more rigorous explanation of the technical design and implementation.

### Classes

#### Base Derivatives

* Motivation: the Base Derivatives class will hold direct computations of primitive derivatives such as logarithmic, exponential, trig, and logistic functions. This will allow us to directly compute derivatives at each stage of the pass. 
* Arguments: each method will take in a function $f$ and output $f'$
* Methods: The methods will include log, exp, trig functions and there inverses, logistic.

#### Dual Number

* Motivation: the Dual Number class will construct the Dual Number objects that will be used in our Forward-Mode.
* Arguments: The `__init__` method will take in an int / float as a first argument for the real part of the Dual Number. The second argument, Dual, will be optional and default to $1$. 
* Methods: This class will have methods that override the Dunder methods of `__add__`, `__pow__`, `__mul__`, `__radd__`, `__rmul__`, `__truediv__`, `__repr__`. This operator overriding will ensure that basic mathematical operations work on Dual Numbers as well as combining them with ints and floats. Critical for the Dunder methods is that they return new Dual Numbers such that they don't return one of the inputs.

#### Node

* Motivation: the Node module holds the Node class that forms the computational graph needed for Reverse-Mode AD. 
* Arguments: The `__init__` method will take in an int / float as a first argument for the value of the node. The second argument, parent_gradients, will be an optional list of tuples where the first element of the tuple is of type Node and the second is an int / float. The third and last argument, name, is of string type and represents the name of the node.
* Methods: Like Dual Number, this class will have methods that override the Dunder methods of `__add__`, `__pow__`, `__mul__`, `__radd__`, `__rmul__`, `__truediv__`, `__repr__`. This operator overriding will ensure that basic mathematical operations work on Nodes as well as combining them with ints and floats. Critical for the Dunder methods is that they return new Nodes such that they don't return one of the inputs.

#### Override Functions

* Motivation: we need to be able to perform basic mathematical computations such as trig functions, exponentials, logarithmic, and logistic using Dual Numbers.
* Arguments: Each function will take as an argument the expression (can be a Dual Number or real number) and output the value after passing the argument through the function.
* Methods: This file will include independent methods such as: `sin(arg)`, `cos(arg)`, `exp(arg)`, `log_base(arg,base)`. These functions will operate on both Dual Numbers, capturing the Dual component as needed, and real numbers.

#### Forward

* Motivation: the Forward-Mode class runs Forward-Mode AD on a function $f$.
* Arguments: the class takes a function `f` in its `__init__` method.
* Methods:
  * compute_derivative accepts an input variable and evaluates the derivative of $f$ at the variable.
  * compute_jacobian computes the Jacobian via computing the derivative on all the input variables.   


#### Reverse 

* Motivation: the Reverse-Mode class runs Reverse-Mode AD on a function $f$. 
* Arguments: the Reverse-Mode class when initialized will accept a function $f$ and a computational graph comprising Nodes.
* Methods: the Reverse-Mode class will have $sort$, get_derivative, get_jacobian, and backward_pass methods. More on these methods in M2: extension.


#### Newton's Method 

* Motivation: the Newton's Method class uses AD to find the roots of a function $f$. 
* Arguments: function $f$, initial guess are required while maximum iterations and tolerance level are optional.
* Methods: the class has only one method, newton, which computes the root over the specified number of iterations which defaults to 200.


### Data Structures

The data structures used will be the computational graph via the Nodes stored in the Graph class. 

### External Libraries
* Numpy will be used for computation using vectors and matrices.
* Pytest will be used for testing our classes.

## Feedback

The feedback received from Milestone 1 was positive overall and our design does not require significant updates. In this revision, we included an additional section in `Implementation` to include the override functions for sin, cos, pow, exp, etc. as suggested in the feedback. The `Software Organization` directory was also updated to reflect this additional file.

## Extensions
To start, copy over the new/future feature section of the documentation from Milestone 2 and update it to reflect the M2 Feedback.
Description of your extension (the feature(s) you implemented in addition to the minimum requirements.)
Additional information or background needed to understand your extension
This could include required mathematics or other concepts

#### Reverse Mode
Our package also handles Reverse Mode autodifferentiation.

Similar to forward mode autodifferentiation, reverse mode autodifferentiation, starts with an input function. It traverses the function by creating new nodes at each elementary operation. For example, take the graph on the left in the image below. In our implementation, we start with the function f(a, c) = (a + b) * a. We start by creating nodes from the input variables, a and b. Next we face an elementary operation: addition. We create a new node, c, that has the attributes value, which is equal to the value of a + the value of b, and the attribute parent_gradients, which is [ (a, 1), (b, 1) ] since it is a list of tuples where the first value is a parent node and the second is the derivative with respect to that node. Next, we face another operation: multiplication. Similarly to above, we create a new node, d, which holds the value a * c and the parents a and c with their respective partial derivatives.

The creation of the tree is the forward pass of Reverse mode, and it happens when we call the `build_node` function on the instance of the Reverse mode class. 

![image](reverse_graph.png)
picture source and algorithm inspiration: https://sidsite.com/posts/autodiff/

The reverse pass is implemented in the `get_gradient` function of the Node class. In the forward pass, the function is saved as the final node in the tree (in the above example, it would be d). Note that it has the value of the whole function evaluated with the given inputs stored in `value`. In the `get_gradient` function, we take that last node and recursively traverse its parents until we get to the input node. Within a given path from the last node to the input node, we multiply the partial derivates at all the edges. Then, we sum the weights of each path and that is the derivative with respect to the input node. To find the jacobian we do this for all input nodes.

Below is another example for finding the jacobian of the function `f(x, y) = xy + e^(xy)` using Reverse mode auto differentiation where x = 1 and y = 2.

![image](reverse_mode.png)

#### Newton's Method

Our package includes a Newton's Method application that enables users to find roots of functions given an initial guess. The application uses AD to compute Jacobian and then on each iteration narrow down to an approximation of the function's roots.

This is used as a sample application of what is possible with autodiffpypi! 



1. Making our software installable via PyPI
- The current project code can be installed and set up per the "Installation" section, which involves a `requirements.txt` file. In the next iteration, it will be distributed via PyPI.

## Broader Impact
Our software provides a tool for users to efficiently compute derivatives and jacobian values. As a public package lisenced under the MIT License, this tool may be used in a number of areas ranging from scientific research, personal projects, and industry applications. While our package provides similar functionality to pre-existing AutoDiff software, releasing our own version still contributes to the public domain by providing a unique implementation and framework that others may find particularly suitable to their use case. 

As the authors of this package, there are a few consequences important to note. Firstly, while our library has undergone unit and integration testing, we cannot guarantee full correctness in all cases, especially under the conditions of large-scale usage. As with most software, this package may be subject to further iteration & fixes. There may be significant consequences if this AutoDiff library is used without user-written tests; therefore we expect any users to take responsibility for ensuring that their applications meet their expectations and that integrating this AutoDiff package does not make them susceptible to incorrect or unexpected results.

## Software Inclusivity
The aim of this package is to increase access to Autodifferentiation capabilities, for students, researchers, programmers, and others who find it applicable. Additionally, our code is released on Github to improve visibility of the internal implementation of such libraries. We welcome open-source contributions or suggestions, which can be submitted by forking this repository and creating a pull-request (this pull request can contain a simple txt file for suggestions). Pull requests will be reviewed and approved by the original authors, who may reach out to you for collaboration. As an international team, we are happy to work with individuals of any background including non-native English speakers, and will provide language accomodation if needed. For those who are visually-impaired, an audio recording of this README can be accessed in the below Google Drive link.

## Future
Automatic differentiation is an extremely powerful tool in a number of scientific fields. In particular, our team is interested in its application to a machine learning context, which is a growing area of research. Much of machine learning research today concerns performance, as developing effective models often requires learning on large datasets. Differentiation is an important tool in this area. It is at the core of optimization algorithms such as gradient descent and stochastic gradient descent that enable such models to train and improve their accuracy. Since most models take in high-input datasets, reverse mode automatic diffferentiation provides an effective way to compute derivatives in only two-passes. In addition, as backpropagation introduces highly complex functions, our package relieves a large degree of computational overhead associated with calculating symbolic derivatives in this case. This motivated our extension of reverse mode in our own AutoDiffPy library. In the future, this area will remain our focus area as we continue to optimize this extension and develop an iterative implementation (as a complement to the current recursive method). We also plan to use our package to develop a simple neural network and release this under the `applications/` folder. This project will act as a guide to users of our library, encouraging further ML and AI development.

## License

Our software will be licensed under the MIT license, which is a highly permissive license and places few restrictions on reuse. As our software project uses standard methods of AD, we do not plan to pursue a patent. Since there are different auto differentiation packages available (such as in NumPy, PyTorch, TensorFlow, MatLab/Fortran (with packages called openAD, ADiMat), C++ (ADLOC), ADiMat, etc.), and we want other software engineers to use ours, we did not want to pick a license that would deter users. We did not add an explicit patent grant because we are not doing anything that has not been done before, so patents are not a consideration for us. Also note that since `numpy` and `pytest` have BSD licenses, we don't need to copy over their licenses.